In [19]:
# pip install scipy


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.8 kB 890.4 kB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/43.6 MB 4.2 MB/s eta 0:00:11
   ---------------------------------------- 0.3/43.6 MB 3.4 MB/s eta 0:00:13
   ---------------------------------------- 0.4/43.6 MB 3.3 MB/s eta 0:00:14
    --------------------------------------- 0.6/43.6 MB 3.5 MB/s eta 0:00:13
    --------------------------------------- 0.7/43.6 MB 3.3 MB/s eta 0:00:13
    --------------------------------------- 0.9/43.6 MB 3.5 MB/s eta 0:00:13
   - -------------------------------------- 1.2/43.6 MB 3.9 MB/s eta 0:00:11
   - -------------------------------------- 1.4/43.6 MB 3.9 MB/s eta 0:00:11
   - ---


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd
import folium
import numpy as np
from folium import plugins
from scipy.spatial.distance import cdist


In [2]:
# Leemos los archivos
df_stops = pd.read_csv("data/cercanias/stops.txt")
df_routes = pd.read_csv("data/cercanias/routes.txt")

In [8]:
# Función para obtener las estaciones de una línea basado en su descripción
def get_line_stations(route_desc):
    stations = route_desc.split('-')
    return [station.strip().upper() for station in stations]

In [21]:
# Función para ordenar las coordenadas de las estaciones
def order_coordinates(coordinates):
    if len(coordinates) <= 2:
        return coordinates
    
    # Comenzamos con el primer punto
    ordered = [coordinates[0]]
    remaining = coordinates[1:]
    
    while remaining:
        # Encontramos el punto más cercano al último punto ordenado
        last_point = np.array([ordered[-1]])
        remaining_points = np.array(remaining)
        
        # Calculamos distancias desde el último punto a todos los restantes
        distances = cdist(last_point, remaining_points).flatten()
        
        # Encontramos el índice del punto más cercano
        nearest_idx = np.argmin(distances)
        
        # Añadimos el punto más cercano a la lista ordenada
        ordered.append(remaining[nearest_idx])
        
        # Eliminamos el punto usado de la lista de restantes
        remaining.pop(nearest_idx)
    
    return ordered

In [9]:
# Creamos un diccionario de estaciones por línea
stations_by_line = {}
for _, route in df_routes.iterrows():
    if pd.notna(route['route_long_name']):
        stations_by_line[route['route_short_name']] = get_line_stations(route['route_long_name'])

In [13]:
# Definimos manualmente las estaciones de cada línea
stations_by_line = {
    'C1': [
        'PRÍNCIPE PÍO', 'PIRÁMIDES', 'DELICIAS', 'MÉNDEZ ÁLVARO', 'ATOCHA', 
        'RECOLETOS', 'NUEVOS MINISTERIOS', 'CHAMARTÍN', 'FUENTE DE LA MORA',
        'VALDEBEBAS', 'AEROPUERTO T4'
    ],
    'C2': [
        'GUADALAJARA', 'AZUQUECA', 'MECO', 'ALCALÁ DE HENARES UNIVERSIDAD',
        'ALCALÁ DE HENARES', 'LA GARENA', 'SOTO DEL HENARES', 'TORREJÓN DE ARDOZ',
        'SAN FERNANDO', 'COSLADA', 'VICÁLVARO', 'SANTA EUGENIA', 'VALLECAS',
        'ATOCHA', 'RECOLETOS', 'NUEVOS MINISTERIOS', 'CHAMARTÍN'
    ],
    'C3': [
        'ARANJUEZ', 'CIEMPOZUELOS', 'VALDEMORO', 'PINTO', 'GETAFE INDUSTRIAL',
        'VILLAVERDE BAJO', 'ATOCHA', 'SOL', 'NUEVOS MINISTERIOS', 'CHAMARTÍN'
    ],
    'C4A': [
        'PARLA', 'GETAFE CENTRO', 'VILLAVERDE ALTO', 'ATOCHA', 'SOL', 
        'NUEVOS MINISTERIOS', 'CHAMARTÍN', 'CANTOBLANCO UNIVERSIDAD', 
        'UNIVERSIDAD P. COMILLAS', 'VALDELASFUENTES', 'ALCOBENDAS-S.S. DE LOS REYES'
    ],
    'C4B': [
        'PARLA', 'GETAFE CENTRO', 'VILLAVERDE ALTO', 'ATOCHA', 'SOL',
        'NUEVOS MINISTERIOS', 'CHAMARTÍN', 'CANTOBLANCO UNIVERSIDAD',
        'EL GOLOSO', 'TRES CANTOS', 'COLMENAR VIEJO'
    ],
    'C5': [
        'MÓSTOLES EL SOTO', 'MÓSTOLES', 'LAS RETAMAS', 'ALCORCÓN',
        'SAN JOSÉ DE VALDERAS', 'CUATRO VIENTOS', 'LAGUNA', 'EMBAJADORES',
        'ATOCHA', 'VILLAVERDE ALTO', 'ZARZAQUEMADA', 'LEGANÉS', 'FUENLABRADA',
        'LA SERNA', 'HUMANES'
    ],
    'C7': [
        'ALCALÁ DE HENARES', 'TORREJÓN DE ARDOZ', 'SAN FERNANDO', 'COSLADA',
        'VICÁLVARO', 'SANTA EUGENIA', 'VALLECAS', 'ATOCHA', 'NUEVOS MINISTERIOS',
        'CHAMARTÍN', 'PRÍNCIPE PÍO'
    ],
    'C8': [
        'GUADALAJARA', 'ALCALÁ DE HENARES', 'TORREJÓN DE ARDOZ', 'ATOCHA',
        'CHAMARTÍN', 'PITIS', 'LAS ROZAS', 'TORRELODONES', 'GALAPAGAR-LA NAVATA',
        'VILLALBA', 'CERCEDILLA'
    ],
    'C9': [
        'CERCEDILLA', 'PUERTO NAVACERRADA', 'COTOS'
    ],
    'C10': [
        'VILLALBA', 'TORRELODONES', 'LAS ROZAS', 'PITIS', 'PRÍNCIPE PÍO',
        'ATOCHA', 'RECOLETOS', 'NUEVOS MINISTERIOS', 'CHAMARTÍN', 'FUENTE DE LA MORA',
        'VALDEBEBAS', 'AEROPUERTO T4'
    ]
}

In [14]:
# Limpiamos los datos de colores
colores_linea = {}
for _, route in df_routes.iterrows():
    if pd.notna(route['route_color']):
        colores_linea[route['route_short_name']] = f"#{route['route_color']}"
    else:
        # Para C4A y C4B usamos colores específicos
        if route['route_short_name'] == 'C4A':
            colores_linea[route['route_short_name']] = '#996633'  # Marrón claro
        elif route['route_short_name'] == 'C4B':
            colores_linea[route['route_short_name']] = '#664422'  # Marrón oscuro

In [24]:
# Creamos el mapa centrado en Madrid
mapa_cercanias = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=11
)

# Procesamos cada línea
for route_name, stations in stations_by_line.items():
    line_group = folium.FeatureGroup(name=f'Línea {route_name}')
    color = colores_linea[route_name]
    
    # Lista para almacenar las coordenadas de las estaciones de esta línea
    line_coordinates = []
    
    # Filtramos las estaciones que pertenecen a esta línea
    for _, stop in df_stops.iterrows():
        if stop['location_type'] == 0:  # Solo paradas, no estaciones padre
            if any(station in stop['stop_name'].upper() for station in stations):
                # Añadimos las coordenadas para la línea
                line_coordinates.append([stop['stop_lat'], stop['stop_lon']])
                
                # Creamos el marcador para la estación
                folium.CircleMarker(
                    location=[stop['stop_lat'], stop['stop_lon']],
                    radius=8,
                    popup=f"""
                        <b>{stop['stop_name']}</b><br>
                        {stop['stop_desc']}<br>
                        Zona: {stop['zone_id']}
                    """,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.2,
                    weight=2,
                    tooltip=stop['stop_name']
                ).add_to(line_group)
    
    # Ordenamos las coordenadas para conectar las estaciones más cercanas
    if len(line_coordinates) > 1:
        ordered_coordinates = order_coordinates(line_coordinates)
        
        # Dibujamos la línea que conecta las estaciones
        folium.PolyLine(
            locations=ordered_coordinates,
            weight=3,
            color=color,
            opacity=0.8
        ).add_to(line_group)
    
    line_group.add_to(mapa_cercanias)

# Añadimos el control de capas
folium.LayerControl(collapsed=False).add_to(mapa_cercanias)

# Añadimos la leyenda
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; 
            left: 50px; 
            width: 150px;
            height: auto;
            z-index: 1000;
            background-color: white;
            padding: 10px;
            border-radius: 5px;
            border: 2px solid grey;
            font-size: 14px;">
    <p style="margin-bottom: 10px;"><strong>Líneas Cercanías</strong></p>
'''

for route_name, color in colores_linea.items():
    legend_html += f'''
    <div style="margin-bottom: 5px;">
        <span style="color: {color}">●</span> {route_name}
    </div>
    '''

legend_html += '</div>'

mapa_cercanias.get_root().html.add_child(folium.Element(legend_html))

In [26]:
# Mostramos el mapa
display(mapa_cercanias)